In [3]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from helper import *

In [4]:
# Parameters
nx = 101
ny = 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) *
    numpy.sin(numpy.pi * X/Lx) *
    numpy.cos(numpy.pi * Y/Ly))

p0 = numpy.zeros((ny,nx))

p_exact = poisson_solution(x, y, Lx, Ly)

In [11]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        return((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
               (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    
    p = p0.copy()
    diff = rtol + 1
    ite = 0
    rk = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # Compute residual rk
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        # Compute sstep size alpha
        Ar[1:-1, 1:-1] = A(rk)
        alpha = numpy.sum(rk * rk) / numpy.sum(rk * Ar)
        # Update Solution
        p = pk + alpha * rk
        # Compute relative difference
        diff = l2_norm(p, pk)
        # Increment interation index
        ite += 1
    return p, ite, diff

In [12]:
p, ites, diff = poisson_2d_steepest_descent(p0, b, dx, dy, rtol=1e-10)
print(ites, diff)

2 1.2374874917997386e-16


In [13]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ad = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    # Compute the initial residual.
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    # Set the initial search direction to be the residual.
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        # Compute the Laplacian of the search direction.
        Ad[1:-1, 1:-1] = A(d)
        # Compute the step size.
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        # Update the solution.
        p = pk + alpha * d
        # Update the residual.
        r = rk - alpha * Ad
        # Update the search direction.
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        # Dirichlet boundary conditions are automatically enforced.
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [15]:
p, ites, diff = poisson_2d_conjugate_gradient(p0, b, dx, dy, rtol=1e-10)
print(ites, diff)

2 [50.00411253811046, 1.2982770796281907e-16]


In [17]:
b_2 = numpy.sin(numpy.pi * X/Lx) * numpy.cos(numpy.pi * Y/Ly) + numpy.sin(6 * numpy.pi * X/Lx) * numpy.cos(6 * numpy.pi * Y/Ly)

In [18]:
p,ites, diff = poisson_2d_conjugate_gradient(p0, b_2, dx, dy, rtol=1e-10)
print(ites, diff)

72 [0.12584130302048865, 0.2912135795524052, 0.19407476010995225, 0.22294206875515055, 0.2965501736991706, 0.3772086938710332, 0.42230864896759046, 0.41054589306069184, 0.35483717922632146, 0.281899799626527, 0.21223685985929855, 0.1551947071493214, 0.11218169760411277, 0.08100101323275162, 0.0586503065427288, 0.04253877488306884, 0.030786896506805838, 0.022142087436103942, 0.015787919297661424, 0.011152726113515945, 0.007481480491069978, 0.003266791726113501, 0.004079286984340981, 0.0034860288633472036, 0.0017435772781646236, 0.0017163080748343752, 0.001729632378582507, 0.0008995005048227992, 0.0009970787962938273, 0.0009642871832898381, 0.0005199651506665473, 0.0007325728292953766, 0.000579801113330843, 0.00043111600295923827, 0.0005771954897007731, 0.00035467363067683715, 0.00044359304713516057, 0.00040128370319751796, 0.00027831070765771973, 0.00035788382853340635, 0.00021164630276973462, 0.00024095193028036854, 0.00018359395985722226, 0.0001346114081130485, 0.00013951838109077863,

In [20]:
p,ites, diff = poisson_2d_steepest_descent(p0, b_2, dx, dy, rtol=1e-10)
print(ites, diff)

20000 3.282250263427408e-08
